In [1]:
import numpy as np
import binary_dude as bd

from numpy import *

import tensorflow as tf
import keras as K
import sys

from keras import layers
from keras import optimizers
from keras import models
from keras import utils
from keras.utils.training_utils import multi_gpu_model

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

K.backend.set_session(session)

In [3]:
# n=1000000
alpha=0.1
delta=0.1
nb_classes=2
L=np.array([[delta, -delta/(1-2*delta), (1-delta)/(1-2*delta)],[delta, (1-delta)/(1-2*delta), -delta/(1-2*delta)]])
L_new=-L+(1-delta)/(1-2*delta)     # A new loss matrix
k_max=40

print L_new

n = 65536
x = bd.bsmc(n, alpha)
z = bd.bsc(x,delta)
Z=utils.np_utils.to_categorical(z,nb_classes)

print x.shape
print z[0:30]

[[1.025 1.25  0.   ]
 [1.025 0.    1.25 ]]
(65536,)
[1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0]


# Neural DUDE with different window size
* 아래 코드는 delta = 0.1 인 상태에서 window size를 달리하여 error rate을 구하는 코드이다.

ipython notebook으로 돌릴 경우

    #k1 = int(sys.argv[1])
    #k2 = int(sys.argv[2])
    #gpu_num = int(sys.argv[3])
    
    k1 = ?
    k2 = ?
    gpu_num = ?
    
위 경우처럼 돌려야 함.

python file 로 돌릴경우

    k1 = int(sys.argv[1])
    k2 = int(sys.argv[2])
    gpu_num = int(sys.argv[3])
    
    #k1 = ?
    #k2 = ?
    #gpu_num = ?
    
python file 실행할 때 argument로 k1, k2, gpu_num을 설정해 주어야함. 아래와 같이 screen 에서 명령어 입력.

    ""CUDA_VISIBLE_DEVICES=? k1 k2 ?""
    CUDE_VISIBLE_DEVICES=0 1 4 0 1 -> 이렇게 돌리면 됨. 
    위와 같이 돌리면 window size = 1~4로 GPU0에 돌린다.
    
    CUDE_VISIBLE_DEVICES=0 1 4 0
    CUDE_VISIBLE_DEVICES=1 5 8 1
    CUDE_VISIBLE_DEVICES=2 9 12 2
    CUDE_VISIBLE_DEVICES=3 13 15 3
    -> 이렇게 4개의 GPU에 4개의 코드를 돌리면 효율적임.
    
* 이렇게 설정한 이유는 여러 GPU에 서로 다른 코드를 돌리기 위함임.
* 그리고 결과는 text file로 저장하세요... 아래와 같이 저장하면 됩니다!

In [5]:
#k1 = int(sys.argv[1])
#k2 = int(sys.argv[2])
#gpu_num = int(sys.argv[3])

k1 = 1
k2 = 4
gpu_num = 0

f = open("NeuralDUDE_k_gpu%d.txt"%gpu_num, "w")
f.write("gpu%d result\n"%gpu_num)
f.close()
for k in xrange(k1, k2+1):
    
    C,Y = bd.make_data_for_ndude(Z,k,L_new,nb_classes,n)

    #--------------------------------------------------------------------------#
    #----------------------------Neural Net model------------------------------#
    inputs = layers.Input(shape = (2 * k * nb_classes,))
    layer = layers.Dense(40, kernel_initializer = 'he_normal')(inputs)
    layer = layers.Activation('relu')(layer)
    layer = layers.Dense(40, kernel_initializer = 'he_normal')(layer)
    layer = layers.Activation('relu')(layer)
    layer = layers.Dense(40, kernel_initializer = 'he_normal')(layer)
    layer = layers.Activation('relu')(layer)
    layer = layers.Dense(3, kernel_initializer = 'he_normal')(layer)
    outputs = layers.Activation('softmax')(layer)
    model = models.Model(inputs = inputs, outputs = outputs)

    adam = optimizers.Adam()
    model.compile(loss='poisson', optimizer=adam)

    model.fit(C,Y,epochs=10,batch_size=100, verbose=0, validation_data=(C, Y))
    #--------------------------------------------------------------------------#
    #--------------------------------------------------------------------------#

    pred_prob = model.predict(C, batch_size = 200, verbose = 0)
    pred_class = np.argmax(pred_prob, axis = 1)
    s_nn_hat=hstack((zeros(k),pred_class,zeros(k)))
    x_nn_hat=bd.denoise_with_s(z,s_nn_hat,k)
    error_nn=bd.error_rate(x,x_nn_hat)
    
    print 'k:%d error_nn=%f'%(k, error_nn)
    
    with open("NeuralDUDE_k_gpu%d.txt"%gpu_num, "a") as f:
        f.write('k:%d error_nn=%f\n'%(k, error_nn))
    

ValueError: invalid literal for int() with base 10: '-f'